# Collect metadata from institutions

This is to form a comparison dataset for looking at items associated with universities not yet using Figshare.

The assumption is that these records go through some type of vetting

In [1]:
import csv
import requests
import json as json
import datetime
import pandas as pd

In [37]:
#List of Institution ids (283-Uni of Cape Town,21-Monash,54-sheffield,898-4tu,797-u Az)
INST_LIST = [283,21,54,898,797]

In [77]:
#Get 50 Figures from each institution
figures = []

for i in INST_LIST:
    query = '{"item_type":1,"institution": ' + str(i) + ',"page": 1, "page_size": 50}' #add more using a comma
    y = json.loads(query) #Figshare API requires json paramaters
    response = requests.post('https://api.figshare.com/v2/articles/search', params=y)
    j=json.loads(response.text) #parse the json into a list named j
    figures.extend(j)
    #for x in j:
    #    figures.append(x) #Above creates a list. So append each record from that list to the master list
    
print(len(figures),'records collected')

156 records collected


In [78]:
#Get 50 Media from each institution
media = []

for i in INST_LIST:
    query = '{"item_type":2,"institution": ' + str(i) + ',"page": 1, "page_size": 50}' #add more using a comma
    y = json.loads(query) #Figshare API requires json paramaters
    response = requests.post('https://api.figshare.com/v2/articles/search', params=y)
    j=json.loads(response.text) #parse the json into a list named j
    media.extend(j)
    
print(len(media),'records collected')

129 records collected


In [79]:
#Get 50 Datasets from each institution
datasets = []

for i in INST_LIST:
    query = '{"item_type":3,"institution": ' + str(i) + ',"page": 1, "page_size": 50}' #add more using a comma
    y = json.loads(query) #Figshare API requires json paramaters
    response = requests.post('https://api.figshare.com/v2/articles/search', params=y)
    j=json.loads(response.text) #parse the json into a list named j
    datasets.extend(j)
    
print(len(datasets),'records collected')

250 records collected


In [80]:
# I don't think this works same as the next one
sampleTest = []
sampleTest = figures + media + datasets
len(sampleTest)

535

In [81]:
sample = figures
sample.extend(media)
sample.extend(datasets)

len(sample)

535

In [84]:
#Save Master File
with open('raw-inst-basic-metadata.json', "w") as write_file:
    json.dump(sample, write_file)

In [83]:
#Create a dataframe from the JSON formatted data
df = pd.DataFrame(sample)

In [ ]:
#Save a file of all the metadata
#save_file = df.to_excel("xxxxxxxxx.xlsx")

In [ ]:
#Open up the same file for reading
with open("raw-inst-basic-metadata.json", "r", encoding='utf8') as read_file: #Replace this with the filename of your choice
    sample = json.load(read_file)

print(len(sample),"records")

# This is where I left off.

In [53]:
#Create list of ids
#article_ids = [item['id'] for item in sample]
article_ids = []
for item in sample:
    article_ids.append(item['id'])
print(len(article_ids))

TypeError: list indices must be integers or slices, not str

In [ ]:
# Create a csv file, use an API to gather data, reopen the csv as a dataframe


#This function deals with multiply nested json elements in the csv write line below
#https://stackoverflow.com/questions/43491287/elegant-way-to-check-if-a-nested-key-exists-in-a-python-dict
def exists(obj, chain):
    _key = chain.pop(0)
    if _key in obj:
        return exists(obj[_key], chain) if chain else obj[_key]

#Create csv file
metadata=open('institution-sample-full-metadata'+str(datetime.datetime.now().strftime("%Y-%m-%d"))+'.csv', 'w', encoding='utf-8', newline='')
#Write header row to csv
csv.writer(metadata).writerow(['id','description','is_embargoed','funding_list','is_metadata_record','published_date','resource_doi','count_categories','count_references','count_tags'])
       
                      
              
for l in sampleid:
    s=requests.get('https://api.figshare.com/v2/articles/'+l)
    r=json.loads(s.text)
    
    #write the values to the csv file. Dates in json files are seconds from jan 1 1970 so datetime.datetime.fromtimestamp converts
    csv.writer(metadata).writerow([
        r['id'],
        r.get('description'), #For any of these .get(), adding ",'N/A'" will fill the null cells with 'N/A'. However, metadata assessment counts non nulls
        r.get('is_embargoed'),
        exists(r,['funding_list']) if exists(r,['funding_list']) !=None else 0,
        r.get('is_metadata_record'),
        r.get('published_date'),
        exists(r,['resource_doi']) if exists(r,['resource_doi']) !=None else 0,
        len(exists(r,['categories'])) if exists(r,['categories'])!=None else 0,
        len(exists(r,['references'])) if exists(r,['references'])!=None else 0,
        len(exists(r,['tags'])) if exists(r,['tags'])!=None else 0]) #write one line to csv file
    
metadata.close() #Close the output file, release all locks

#Open up the same file as a dataframe. Encode cp1252 avoids a utf8 error.
dffull = pd.read_csv('institution-sample-full-metadata'+str(datetime.datetime.now().strftime("%Y-%m-%d"))+'.csv',encoding='utf8')

print('The resulting dataframe has',len(dffull),' rows')

In [ ]:
#Deal with funders 
funderMaster = [] #funder name might be duplicated across multiple grants for an item. Create a dictionary to deduplicate later
for f in sample['funding_list']:
    entry = {"id":article_id, "funder_name":f['funder_name']}
    funderMaster.append(entry)

In [ ]:
#Save funder File
with open('funder-name.json', "w") as write_file:
    json.dump(funderMaster, write_file)